In [1500]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1501]:
# Load the data
df = pd.read_csv("all_data.csv", encoding="utf-8")
df.head()


,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode,categories
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,"['access_limited', 'access_limited.private', '..."
1,-16236108,Ballenoil,40.373971,-3.776696,Spain,Community of Madrid,Madrid,NaN,NaN,Latina,Calle de la Sinfonía,28917,"['building', 'building.commercial', 'service',..."
2,-15931847,Bluespace,40.373500,-3.642409,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Calle Gamonal,28018,"['commercial', 'rental', 'rental.storage']"
3,-15931821,La parrilla del sur,40.371670,-3.639826,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Avenida de Moratilla de los Meleros,28031,"['building', 'building.catering', 'catering', ..."
4,-15931820,Maxicasa,40.371721,-3.639581,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Carretera de Villaverde a Vallecas,28031,"['building', 'building.commercial', 'commercia..."


In [1502]:
# Split the categories into a list, remove brackets from the strings that have them and the single quotation marks too
df["categories"] = df["categories"].str.replace("[", "").str.replace("]", "").str.replace("'","").str.split(", ")
df_cats = df.explode("categories")
print(df_cats.categories)

0                     access_limited
0             access_limited.private
0                            service
0                  service.recycling
0           service.recycling.centre
                    ...             
21884            catering.restaurant
21884    catering.restaurant.italian
21884      catering.restaurant.pizza
21885                       catering
21885             catering.fast_food
Name: categories, Length: 61342, dtype: object


In [1503]:
#Return a list of all the unique categories
print(df_cats.categories.unique())

categories = df_cats.categories.unique()

['access_limited' 'access_limited.private' 'service' 'service.recycling'
 'service.recycling.centre' 'building' 'building.commercial'
 'service.vehicle' 'service.vehicle.fuel' 'commercial' 'rental'
 'rental.storage' 'building.catering' 'catering' 'catering.restaurant'
 'commercial.houseware_and_hardware' 'building.industrial'
 'service.vehicle.repair' 'service.vehicle.repair.car'
 'commercial.elektronics' 'commercial.trade' 'building.office' 'office'
 'office.government' 'wheelchair' 'wheelchair.yes'
 'commercial.outdoor_and_sport' 'commercial.shopping_mall'
 'commercial.supermarket' 'building.facility' 'service.police'
 'building.public_and_civil' 'production' 'production.factory'
 'service.social_facility' 'office.educational_institution'
 'office.government.ministry' 'office.government.social_security'
 'office.foundation' 'service.financial' 'service.financial.bank'
 'office.company' 'office.research' 'building.historic'
 'office.government.administrative' 'tourism' 'tourism.sights

In [1504]:
df_cats

,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode,categories
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,access_limited
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,access_limited.private
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,service
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,service.recycling
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,service.recycling.centre
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21884,11835944044,Eccolo,40.409095,-3.705602,Spain,Community of Madrid,Madrid,Centro,Lavapiés,Embajadores,Calle de Embajadores,28012,catering.restaurant
21884,11835944044,Eccolo,40.409095,-3.705602,Spain,Community of Madrid,Madrid,Centro,Lavapiés,Embajadores,Calle de Embajadores,28012,catering.restaurant.italian
21884,11835944044,Eccolo,40.409095,-3.705602,Spain,Community of Madrid,Madrid,Centro,Lavapiés,Embajadores,Calle de Embajadores,28012,catering.restaurant.pizza
21885,11836002693,Pecados Argetinos,40.409834,-3.707097,Spain,Community of Madrid,Madrid,Centro,Lavapiés,NaN,Calle Ribera de Curtidores,28005,catering


In [1505]:
#Create a dictionary with all the main categories (before the dot) as keys and a list of all the subcategories as values
categories_dict = {}
for cat in categories:
    main_cat = cat.split(".")[0]
    #Throw out the categories that have more than one dot
    if len(cat.split(".")) > 2:
        continue
    if main_cat in categories_dict:
        categories_dict[main_cat].append(cat)
    else:
        categories_dict[main_cat] = [cat]
#From each key in the dictionary, remove all the values that do not have a period in them
for key in categories_dict:
    categories_dict[key] = [cat for cat in categories_dict[key] if "." in cat]

In [1506]:
new_cats_dict = {k: v for k, v in categories_dict.items() if k in ["commercial", "production", "office", "service", "catering"]}
#print the number of values of all the values as keys
print({k: len(v) for k, v in new_cats_dict.items()})

{'service': 15, 'commercial': 41, 'catering': 9, 'office': 31, 'production': 4}


In [1507]:
new_cats_dict["commercial"]

['commercial.houseware_and_hardware',
 'commercial.elektronics',
 'commercial.trade',
 'commercial.outdoor_and_sport',
 'commercial.shopping_mall',
 'commercial.supermarket',
 'commercial.marketplace',
 'commercial.department_store',
 'commercial.tickets_and_lottery',
 'commercial.furniture_and_interior',
 'commercial.books',
 'commercial.convenience',
 'commercial.garden',
 'commercial.vehicle',
 'commercial.health_and_beauty',
 'commercial.florist',
 'commercial.smoking',
 'commercial.food_and_drink',
 'commercial.kiosk',
 'commercial.clothing',
 'commercial.hobby',
 'commercial.toy_and_game',
 'commercial.discount_store',
 'commercial.newsagent',
 'commercial.pet',
 'commercial.gift_and_souvenir',
 'commercial.stationery',
 'commercial.jewelry',
 'commercial.bag',
 'commercial.chemist',
 'commercial.art',
 'commercial.erotic',
 'commercial.watches',
 'commercial.second_hand',
 'commercial.video_and_music',
 'commercial.antiques',
 'commercial.gas',
 'commercial.baby_goods',
 'commer

In [1508]:
new_cats_dict["production"]

['production.factory',
 'production.brewery',
 'production.pottery',
 'production.winery']

In [1509]:
new_cats_dict["office"]

['office.government',
 'office.educational_institution',
 'office.foundation',
 'office.company',
 'office.research',
 'office.diplomatic',
 'office.insurance',
 'office.political_party',
 'office.employment_agency',
 'office.non_profit',
 'office.estate_agent',
 'office.association',
 'office.financial',
 'office.it',
 'office.notary',
 'office.energy_supplier',
 'office.coworking',
 'office.lawyer',
 'office.charity',
 'office.security',
 'office.travel_agent',
 'office.architect',
 'office.tax_advisor',
 'office.accountant',
 'office.religion',
 'office.newspaper',
 'office.telecommunication',
 'office.consulting',
 'office.advertising_agency',
 'office.logistics',
 'office.financial_advisor']

In [1510]:
new_cats_dict["service"]

['service.recycling',
 'service.vehicle',
 'service.police',
 'service.social_facility',
 'service.financial',
 'service.funeral_directors',
 'service.post',
 'service.beauty',
 'service.estate_agent',
 'service.taxi',
 'service.travel_agency',
 'service.cleaning',
 'service.bookmaker',
 'service.tailor',
 'service.locksmith']

In [1511]:
new_cats_dict["catering"]

['catering.restaurant',
 'catering.pub',
 'catering.fast_food',
 'catering.bar',
 'catering.cafe',
 'catering.taproom',
 'catering.biergarten',
 'catering.ice_cream',
 'catering.food_court']

In [1512]:
#I need to load the data again into another dataframe

df2 = pd.read_csv("all_data.csv", encoding="utf-8")
df4 = df2.copy()
df2.head()

,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode,categories
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052,"['access_limited', 'access_limited.private', '..."
1,-16236108,Ballenoil,40.373971,-3.776696,Spain,Community of Madrid,Madrid,NaN,NaN,Latina,Calle de la Sinfonía,28917,"['building', 'building.commercial', 'service',..."
2,-15931847,Bluespace,40.373500,-3.642409,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Calle Gamonal,28018,"['commercial', 'rental', 'rental.storage']"
3,-15931821,La parrilla del sur,40.371670,-3.639826,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Avenida de Moratilla de los Meleros,28031,"['building', 'building.catering', 'catering', ..."
4,-15931820,Maxicasa,40.371721,-3.639581,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Carretera de Villaverde a Vallecas,28031,"['building', 'building.commercial', 'commercia..."


In [1513]:
df4.isnull().sum()

id                   0
name                 0
latitude             0
longitude            0
country              0
state                0
city                 0
district         12803
neighbourhood    15772
suburb             988
street              20
postcode             0
categories           0
dtype: int64

In [1514]:
print(df4.count())

id               21886
name             21886
latitude         21886
longitude        21886
country          21886
state            21886
city             21886
district          9083
neighbourhood     6114
suburb           20898
street           21866
postcode         21886
categories       21886
dtype: int64


In [1515]:
#Remove the category list because the final dataframe will look different
df3 = df2.drop(columns="categories")

df3.head()

,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode
0,-17027791,Almacén de la villa,40.393615,-3.600524,Spain,Community of Madrid,Madrid,NaN,NaN,Vicálvaro,Calle Dehesa Vieja,28052
1,-16236108,Ballenoil,40.373971,-3.776696,Spain,Community of Madrid,Madrid,NaN,NaN,Latina,Calle de la Sinfonía,28917
2,-15931847,Bluespace,40.373500,-3.642409,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Calle Gamonal,28018
3,-15931821,La parrilla del sur,40.371670,-3.639826,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Avenida de Moratilla de los Meleros,28031
4,-15931820,Maxicasa,40.371721,-3.639581,Spain,Community of Madrid,Madrid,NaN,NaN,Villa de Vallecas,Carretera de Villaverde a Vallecas,28031


In [1516]:
#From df2.categories, remove the brackets and the single quotation marks and split the strings into lists.
df2["categories"] = df2["categories"].str.replace("[", "").str.replace("]", "").str.replace("'","").str.split(", ")

#in the categories column, only keep those in the new_cats_dict keys or values
df2["categories"] = df2["categories"].apply(lambda x : [cat for cat in x if (any([cat in new_cats_dict[key] for key in new_cats_dict])) or (any ([cat in new_cats_dict]))])

#Create a new column for each key in the new_cats_dict and assign 1 if any of the values in the list are in the categories column, 0 otherwise

df_new_cats = pd.DataFrame()

for key in new_cats_dict:
    df_new_cats[key] = df2["categories"].apply(lambda x: 1 if any([cat in x for cat in new_cats_dict[key]]) else 0)

df_new_cats.head()

,service,commercial,catering,office,production
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,0,0,1,0,0
4,0,1,0,0,0


In [1517]:
#Create a new column for each key in the new_cats_dict and assign a string with all the subcategories that are in the categories column
for key in new_cats_dict:    
    df_new_cats[key + "_subcats"] = df2["categories"].apply(lambda x: str([cat for cat in x if cat in new_cats_dict[key] and cat != key]).replace("[", "").replace("]", "").replace("'",""))
    df_new_cats[key + "_subcats"] = df_new_cats[key + "_subcats"].apply(lambda x: x if x != "" else None)
#Fill empty spaces with zero
df_new_cats = df_new_cats.fillna(0)
df_new_cats.head()

,service,commercial,catering,office,production,service_subcats,commercial_subcats,catering_subcats,office_subcats,production_subcats
0,1,0,0,0,0,service.recycling,0,0,0,0
1,1,0,0,0,0,service.vehicle,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,catering.restaurant,0,0
4,0,1,0,0,0,0,commercial.houseware_and_hardware,0,0,0


In [1518]:
#create a column for every subcategory in the new_cats_dict and assign 1 if the subcategory is in the categories column, 0 otherwise
df_subcats = pd.DataFrame()

for key in new_cats_dict:
    for cat in new_cats_dict[key]:
        df_subcats[cat] = df2["categories"].apply(lambda x: 1 if cat in x else 0)

df_clean_data = pd.concat([df3, df_new_cats, df_subcats], axis=1)

df_clean_data.head()

#Save the data
df_clean_data.to_csv("clean_data.csv", index=False)